In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
path_source = 'D:\sen12floods\sen12floods_s1_source'
path_labels = 'D:\sen12floods\sen12floods_s1_labels'

os.chdir(path_source)

# use loop to gather all files from directory
images = []
for root, curDir, files in os.walk(path_source):
    for file in files:
        images.append(os.path.join(root,file))


paths_source = []
for imgPath in images:
    if imgPath.endswith(".tif"):
        paths_source.append(imgPath) # gather all tif file paths

        
os.chdir(path_labels)

# use loop to gather all json files from directory
labels = []
for root, curDir, files in os.walk(path_labels):
    for file in files:
        labels.append(os.path.join(root,file))
        
paths_labels = []
for labelPath in labels:
    if labelPath.endswith(".json"):
        paths_labels.append(labelPath) # gather all .json file paths
# extraxt id from 'source'

IDs=[]
for i in paths_source:
    source=os.path.split(os.path.split(i)[0])[1]
    m='_'.join(source.split('_')[3:])
    IDs.append(m)

idDict=dict(zip(paths_source, IDs))

import json
# extraxt label from label

label=[]
key=[]
for i in paths_labels:
    with open(i) as f:
        data=json.load(f)
        label_id=data["id"]
        k=data["properties"]["FLOODING"]
        label.append(k)
        l='_'.join(label_id.split('_')[3:])
        key.append(l)

myDict=dict(zip(key,label))


flooding=[]
non_flooding=[]
labels=[]
# if date of source matches that of label, check label(0 or 1)
# if 0, store in non-flooding; if 1, store in flooding

for i in idDict:
    for j in myDict:
        if j==idDict[i]:
            labels.append(myDict[j])
            if myDict[j]=='True':
                flooding.append(i)
            else:
                non_flooding.append(i)

In [3]:
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.preprocessing import normalize
from PIL import Image
import tifffile

ims_flooding = []
for im in flooding:
    image=tifffile.imread(im)
    image_resized=resize(image, (224, 224), anti_aliasing=True)
    ims_flooding.append(normalize(image_resized))
    
    ims_nflooding = []
for im in non_flooding:
    image=tifffile.imread(im)
    image_resized=resize(image, (224, 224), anti_aliasing=True)
    ims_nflooding.append(normalize(image_resized))
    
    # gather all images to a list:
full_data=[]
full_data=ims_flooding + ims_nflooding

# all labels to a list:
targets=[]
label_train_f=[1]*len(ims_flooding)
label_train_nf=[0]*len(ims_nflooding)
targets=label_train_f + label_train_nf

# split into 70% training set and 30% testing set
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    full_data, targets, test_size=0.3, random_state=42)

train_X, val_X, train_y, val_y = train_test_split(
    data_train, target_train, test_size=0.2, random_state=42)


In [5]:
# list to array 
#for training set
cnn_train_X=np.array(train_X)
cnn_train_y=np.array(train_y)

# for validation set
cnn_validation_X=np.array(val_X)
cnn_validation_y=np.array(val_y)

# for test set
cnn_testn_X=np.array(data_test)
cnn_test_y=np.array(target_test)

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((cnn_train_X, train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

In [9]:
# data to rgb
rgb_batch_train = np.repeat(cnn_train_X[..., np.newaxis], 3, -1)
rgb_batch_validation = np.repeat(cnn_validation_X[..., np.newaxis], 3, -1)
rgb_batch_test = np.repeat(cnn_testn_X[..., np.newaxis], 3, -1)

In [30]:
# resize labels to 2d array [batch, label]

y_train = np.asarray(cnn_train_y).astype('float32').reshape((-1,1))
y_validation = np.asarray(cnn_validation_y).astype('float32').reshape((-1,1))
y_test = np.asarray(cnn_test_y).astype('float32').reshape((-1,1))

In [32]:
print(cnn_train_X.shape, cnn_validation_X.shape, cnn_testn_X.shape)
print(rgb_batch_train.shape, rgb_batch_validation.shape, rgb_batch_test.shape)
print(y_train.shape, y_validation.shape, y_test.shape)

(3731, 224, 224) (933, 224, 224) (1999, 224, 224)
(3731, 224, 224, 3) (933, 224, 224, 3) (1999, 224, 224, 3)
(3731, 1) (933, 1) (1999, 1)


In [93]:
# load pretrained model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# # # build a sequential model
# resnet_model=Sequential()

# load pretrained model using ResNet50
resnet= tf.keras.applications.ResNet50(
                   include_top=False,
                   input_shape=(224, 224, 3),
                    pooling='max',
                   weights='imagenet')

x=resnet.layers[-6].output
# output=Dense(units=2, activation='sigmoid')(x)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model=Model(inputs=resnet.input, outputs=outputs)

# freeze all layers except for teh last 30 layers
for layer in model.layers[:-30]:
        layer.trainable=False

# # add pretrained resnet model to sequential model
# resnet_model.add(pretrained_model)

In [94]:
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.layers import Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D

# add dropout layers to prevent overfitting

# resnet_model.add(GlobalAveragePooling2D())
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dense(256, activation='relu'))
# resnet_model.add(Dropout(0.2))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dense(128, activation='relu'))
# resnet_model.add(Dropout(0.2))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dense(64, activation='relu'))
# resnet_model.add(Dropout(0.2))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dense(1, activation='sigmoid'))

# inputs = keras.Input(shape=(224, 224, 3))

# pretrained_model.trainable = False

# inputs = keras.layers.GlobalAveragePooling2D()(inputs)
# inputs = keras.layers.Dropout(0.2)(inputs)  # Regularize with dropout
# outputs = keras.layers.Dense(1)(inputs)

# model = keras.Model(inputs, outputs)

model.summary()


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_21[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [95]:
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD

opt=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# train the top layer
model.compile(
    optimizer=opt,
    loss='BinaryCrossentropy',
    metrics=['accuracy'])

# def lr_step_decay(epoch, lr):
#     drop_rate=0.5
#     epochs_drop=1
#     return 0.001*math.pow(drop_rate, math.floor(epoch/epoch_drop)) 

model.fit(rgb_batch_train, y_train, 
          batch_size=32, 
          epochs=10, 
          validation_data=(rgb_batch_validation, y_validation)
#                  callbacks=[LearningRateScheduler(lr_step_decay)]
         )
# sources of code: https://medium.com/@sohailahmedkhan173/flood-detection-in-images-using-keras-a-tutorial-on-transfer-learning-fine-tuning-db22011af67c
# sources of code: https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/
# more sources: https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b

Epoch 1/10
117/117 [==============================] - 733s 5s/step - loss: 0.5606 - accuracy: 0.6990 - val_loss: 0.6425 - val_accuracy: 0.6902
Epoch 2/10
117/117 [==============================] - 433s 4s/step - loss: 0.5255 - accuracy: 0.7095 - val_loss: 0.6098 - val_accuracy: 0.6902
Epoch 3/10
117/117 [==============================] - 359s 3s/step - loss: 0.5102 - accuracy: 0.7175 - val_loss: 0.5449 - val_accuracy: 0.6892
Epoch 4/10
117/117 [==============================] - 322s 3s/step - loss: 0.4995 - accuracy: 0.7234 - val_loss: 0.5803 - val_accuracy: 0.6302
Epoch 5/10
117/117 [==============================] - 321s 3s/step - loss: 0.4861 - accuracy: 0.7349 - val_loss: 0.5756 - val_accuracy: 0.6902
Epoch 6/10
117/117 [==============================] - 332s 3s/step - loss: 0.4796 - accuracy: 0.7373 - val_loss: 0.6980 - val_accuracy: 0.6892
Epoch 7/10
117/117 [==============================] - 336s 3s/step - loss: 0.4657 - accuracy: 0.7432 - val_loss: 0.7935 - val_accuracy: 0.6892

In [96]:
# continue training with more epochs

model.fit(rgb_batch_train, y_train, 
          batch_size=32, 
          epochs=5, 
          validation_data=(rgb_batch_validation, y_validation)
         )

Epoch 1/5
117/117 [==============================] - 537s 5s/step - loss: 0.4446 - accuracy: 0.7671 - val_loss: 0.4765 - val_accuracy: 0.7417
Epoch 2/5
117/117 [==============================] - 415s 4s/step - loss: 0.4397 - accuracy: 0.7695 - val_loss: 0.4826 - val_accuracy: 0.7503
Epoch 3/5
117/117 [==============================] - 334s 3s/step - loss: 0.4247 - accuracy: 0.7840 - val_loss: 1.1047 - val_accuracy: 0.6902
Epoch 4/5
117/117 [==============================] - 335s 3s/step - loss: 0.4347 - accuracy: 0.7727 - val_loss: 2.1729 - val_accuracy: 0.6902
Epoch 5/5
117/117 [==============================] - 323s 3s/step - loss: 0.4251 - accuracy: 0.7869 - val_loss: 0.4950 - val_accuracy: 0.7546


In [97]:
# more epochs..seems to have more room for improvements
model.fit(rgb_batch_train, y_train, 
          batch_size=32, 
          epochs=5, 
          validation_data=(rgb_batch_validation, y_validation)
         )

Epoch 1/5
117/117 [==============================] - 697s 6s/step - loss: 0.4238 - accuracy: 0.7858 - val_loss: 2.6304 - val_accuracy: 0.6902
Epoch 2/5
117/117 [==============================] - 356s 3s/step - loss: 0.4081 - accuracy: 0.7960 - val_loss: 1.2620 - val_accuracy: 0.4587
Epoch 3/5
117/117 [==============================] - 341s 3s/step - loss: 0.4059 - accuracy: 0.7912 - val_loss: 1.9569 - val_accuracy: 0.3537
Epoch 4/5
117/117 [==============================] - 348s 3s/step - loss: 0.3988 - accuracy: 0.8043 - val_loss: 2.9478 - val_accuracy: 0.3355
Epoch 5/5
117/117 [==============================] - 334s 3s/step - loss: 0.4056 - accuracy: 0.7963 - val_loss: 1.3397 - val_accuracy: 0.6902


In [99]:
print(history.history.keys())

NameError: name 'history' is not defined

In [98]:
# fig1 = plt.gcf()
# plt.plot(model.history['accuracy'])
# plt.plot(model.history['val_accuracy'])
# plt.axis(ymin=0.4,ymax=1)
# plt.grid()
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(['train', 'validation'])
# plt.show()

TypeError: 'History' object is not subscriptable

<Figure size 640x480 with 0 Axes>

In [100]:
#unfreeze pretrained model for fine tuning
resnet.trainable=True

model.compile(    
    optimizer=opt,
    loss='BinaryCrossentropy',
    metrics=['accuracy'])

model.fit(rgb_batch_train, y_train, 
          batch_size=32, 
          epochs=10, 
          validation_data=(rgb_batch_validation, y_validation)
         )

Epoch 1/10
117/117 [==============================] - 1826s 11s/step - loss: 0.4810 - accuracy: 0.7590 - val_loss: 0.6378 - val_accuracy: 0.6902
Epoch 2/10
117/117 [==============================] - ETA: 0s - loss: 0.3549 - accuracy: 0.8464

KeyboardInterrupt: 

In [ ]:
model.fit(rgb_batch_test, y_test)